In [85]:
%load_ext lab_black

from puzzles import load

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [86]:
from toolz import groupby
from datetime import datetime
import numpy as np

In [87]:
data = load(4)

# data = """[1518-11-01 00:00] Guard #10 begins shift
# [1518-11-01 00:05] falls asleep
# [1518-11-01 00:25] wakes up
# [1518-11-01 00:30] falls asleep
# [1518-11-01 00:55] wakes up
# [1518-11-01 23:58] Guard #99 begins shift
# [1518-11-02 00:40] falls asleep
# [1518-11-02 00:50] wakes up
# [1518-11-03 00:05] Guard #10 begins shift
# [1518-11-03 00:24] falls asleep
# [1518-11-03 00:29] wakes up
# [1518-11-04 00:02] Guard #99 begins shift
# [1518-11-04 00:36] falls asleep
# [1518-11-04 00:46] wakes up
# [1518-11-05 00:03] Guard #99 begins shift
# [1518-11-05 00:45] falls asleep
# [1518-11-05 00:55] wakes up"""

lines = data.strip().split("\n")
lines = sorted(
    lines, key=lambda a: datetime.strptime(a.split("] ")[0], "[%Y-%m-%d %H:%M")
)

In [88]:
def parse_event(line: str):
    dtime, event = line.split("] ")
    d = datetime.strptime(dtime, "[%Y-%m-%d %H:%M")
    return d, event


def get_minute(d):
    return d.hour * 60 + d.minute


def create_new_nightshift_if_needed(data, idx, date):
    if (d.month, d.day) not in data[idx]:
        # выделяем 24 часа на ночь в этот день, — мало ли?)
        day_vec = np.zeros(12 * 60, dtype=np.uint8)
        data[idx][(d.month, d.day)] = day_vec
    # Из-за того, что словарь — mutable структура данных,
    # функция ничего не возвращает, — изменения в словаре будут видны и снаружи


guards_days = {}


for line in lines:
    d, event = parse_event(line)

    # Если это строчка про охранника,
    if event.startswith("Guard"):

        # сделали для него свой словарь, если встретили впервые
        id_guard = int(event.split()[1][1:])
        if id_guard not in guards_days:
            guards_days[id_guard] = {}

        # Сразу сделали ему ночь в расчете на то, что он никогда не спит
        create_new_nightshift_if_needed(guards_days, id_guard, d)

        # и на следующей итерации в id_guard уже будет лежать его id.

    # Если время засыпать,
    elif event.startswith("falls asleep"):

        # (если эта запись про новый день — выделяем новую ночную смену)
        create_new_nightshift_if_needed(guards_days, id_guard, d)

        # запоминаем время, когда он засыпает.
        start = get_minute(d)

    # Если время просыпаться,
    elif event.startswith("wakes up"):

        # вычисляем минуту, когда проснется,
        end = get_minute(d)

        # и отмечаем на графике сна промежуток.
        guards_days[id_guard][(d.month, d.day)][start:end] = 1

*Из любопытного:* в процессе формирования такой структуры данных нам были нужны отметки о месяце и дне, чтобы случайно не перепутать ночи дежурств, но после того, как мы сделали наш мега-словарь, они не нужны. Давайте избавимся от них, просто сделав матрицы из векторов-ночей:

In [89]:
guards_days = {
    id_guard: np.stack(days.values()) for id_guard, days in guards_days.items()
}

## Первая часть задачи

In [90]:
s_max = -1
for k in guards_days:
    s = guards_days[k].sum()
    if s > s_max:
        s_max = s
        k_best = k

v = guards_days[k_best].sum(axis=0)
best_minute = np.arange(len(v))[v == v.max()][-1]
best_minute, k_best, k_best * best_minute

(44, 3203, 140932)

## Вторая часть задачи

In [91]:
max_sleeps = {k: v.sum(axis=0) for k, v in guards_days.items()}

v_max = -1
for k, v in max_sleeps.items():
    if v.max() > v_max:
        v_max = v.max()
        k_best = k

v = max_sleeps[k_best]
best_minute = np.arange(len(v))[v == v_max][-1]

best_minute, k_best, k_best * best_minute

(32, 1601, 51232)